In [4]:
! pip3 install Pillow


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
from PIL import Image
from PIL.ExifTags import TAGS

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

imagename = "ImagesPokemon/abra.png"

image = Image.open(imagename)

In [6]:
info_dict = {
    "Filename": image.filename,
    "Image Size": image.size,
    "Image Height": image.height,
    "Image Width": image.width,
    "Image Format": image.format,
    "Image Mode": image.mode,
    "Image is Animated": getattr(image, "is_animated", False),
    "Frames in Image": getattr(image, "n_frames", 1)
}

for label,value in info_dict.items():
    print(f"{label:25}: {value}")

Filename                 : ImagesPokemon/abra.png
Image Size               : (120, 120)
Image Height             : 120
Image Width              : 120
Image Format             : PNG
Image Mode               : RGBA
Image is Animated        : False
Frames in Image          : 1


In [7]:
def convert_P_to_RGBA():
    dossier = "ImagesPokemon"
    for filename in os.listdir(dossier):
        imagename = os.path.join(dossier, filename)

        if os.path.isfile(imagename):
            image = Image.open(imagename)
            if image.mode == 'P':
                image2 = image.convert('RGBA')
                image2.save(image.filename.replace("ImagesPokemon/","ImagesPokemonPASP/"))
    return "Ok"

In [8]:
import os
i=0
dossier = "ImagesPokemon"
for filename in os.listdir(dossier):
    imagename = os.path.join(dossier, filename)
    if os.path.isfile(imagename):
        image = Image.open(imagename)
    
    info_dict = {
    "Filename": image.filename,
    "Image Size": image.size,
    "Image Height": image.height,
    "Image Width": image.width,
    "Image Format": image.format,
    "Image Mode": image.mode,
    "Image is Animated": getattr(image, "is_animated", False),
    "Frames in Image": getattr(image, "n_frames", 1)
    }

    for label,value in info_dict.items():
        print(f"{label:25}: {value}")

Filename                 : ImagesPokemon/parasect.png
Image Size               : (120, 120)
Image Height             : 120
Image Width              : 120
Image Format             : PNG
Image Mode               : RGBA
Image is Animated        : False
Frames in Image          : 1
Filename                 : ImagesPokemon/lumineon.png
Image Size               : (120, 120)
Image Height             : 120
Image Width              : 120
Image Format             : PNG
Image Mode               : RGBA
Image is Animated        : False
Frames in Image          : 1
Filename                 : ImagesPokemon/raikou.png
Image Size               : (120, 120)
Image Height             : 120
Image Width              : 120
Image Format             : PNG
Image Mode               : RGBA
Image is Animated        : False
Frames in Image          : 1
Filename                 : ImagesPokemon/comfey.jpg
Image Size               : (120, 120)
Image Height             : 120
Image Width              : 120
Image Format 

In [9]:
from PIL import Image
import numpy
import math
import matplotlib.pyplot as plot
import pandas as pd
from sklearn.cluster import MiniBatchKMeans
from scipy.spatial import KDTree
from webcolors import (CSS3_HEX_TO_NAMES, hex_to_rgb,)

"function that convert RGB numbers into a color name"
def rgb_to_name(rgb):
    # a disctionnary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb)
    return f'{names[index]}'

#En dessous, plot et pie des couleurs prédominantes

def find_predominant_color(imagepath):

    nombre_couleurs = 5 #Un de plus que ce qu'on veux

    imgfile = Image.open(imagepath)
    numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    clusters = MiniBatchKMeans(n_clusters=nombre_couleurs)
    clusters.fit(numarray)
    npbins = numpy.arange(0, nombre_couleurs+1)
    histogram = numpy.histogram(clusters.labels_, bins=npbins)
    [list1,list2] = histogram
    list1,list2 = zip(*sorted(zip(list1,list2),reverse=True))
    histogram_sort = [list1,list2]

    labels = numpy.unique(clusters.labels_)
    #barlist = plot.bar(labels[1:], histogram_sort[0][1:])
    list_colors = []
    colornames = []
    for i in range(nombre_couleurs-1):
        list_colors.append(
            "#%02x%02x%02x"
            % (
                math.ceil(clusters.cluster_centers_[list2[i+1]][0]),
                math.ceil(clusters.cluster_centers_[list2[i+1]][1]),
                math.ceil(clusters.cluster_centers_[list2[i+1]][2]),
            )
        )
        #barlist[i].set_color(
        #    "#%02x%02x%02x"
        #    % (
        #        math.ceil(clusters.cluster_centers_[list2[i+1]][0]),
        #        math.ceil(clusters.cluster_centers_[list2[i+1]][1]),
        #        math.ceil(clusters.cluster_centers_[list2[i+1]][2]),
        #    )
        #)
        rgb = [math.ceil(clusters.cluster_centers_[list2[i+1]][0]),
                math.ceil(clusters.cluster_centers_[list2[i+1]][1]),
                math.ceil(clusters.cluster_centers_[list2[i+1]][2]),]
        colornames.append(rgb_to_name(rgb))
    #plot.show()
    #fig,ax = plot.subplots()
    #ax.pie(histogram_sort[0][1:], startangle=90, colors=list_colors)
    #ax.axis('equal')
    #plot.show()

    return colornames

In [10]:
import json

def write_json(filename, data):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

import os
toutes_les_infos = []
df = pd.read_csv('All_Pokemon.csv', sep=',')
dossier = "ImagesPokemon"
for filename in os.listdir(dossier):
    imagename = os.path.join(dossier, filename)
    if os.path.isfile(imagename):
        image = Image.open(imagename)
        couleur = find_predominant_color(imagename)

        for i in range(len(df)):
            if df["Name"][i].lower() == image.filename.split(".")[0].split("/")[1]:
                #type2 = df["Type 2"][i]
                #if type2 == "NaN":
                #    type2 = "None"
                info_dict = {
                "Filename": image.filename,
                "Name" : image.filename.split(".")[0].split("/")[1].capitalize(),
                "Image Size": image.size,
                "Image Height": image.height,
                "Image Width": image.width,
                "Image Format": image.format,
                "Image Mode": image.mode,
                "Image is Animated": getattr(image, "is_animated", False),
                "Frames in Image": getattr(image, "n_frames", 1),
                "Predominant color": [
                    couleur[0],
                    couleur[1],
                    couleur[2],
                    couleur[3]
                ],
                "Type 1": df["Type 1"][i],
                #"Type 2": type2,
                "Generation": df["Generation"][i],
                "Legendary": df["Legendary"][i],
                "Mega Evolution": df["Mega Evolution"][i],
                "Height": df["Height"][i],
                "Weight": df["Weight"][i],
                }
                toutes_les_infos.append(info_dict)
                break

write_json("pokemonjson.json", toutes_les_infos)

In [12]:
dictionnaire_preferences = {} #On va mettre les préférences de chaque utilisateur dans un dictionnaire

def get_tags():
    tags = []
    with open('pokemonjson.json', 'r') as f:
        jason = json.load(f)
    for pokemon in jason:
        tags.append([pokemon["Filename"].split(".")[0].split("/")[1].capitalize(),pokemon["Predominant color"][0],pokemon["Predominant color"][1],pokemon["Predominant color"][2],pokemon["Predominant color"][3],pokemon["Type 1"],pokemon["Generation"],pokemon["Legendary"],pokemon["Mega Evolution"],pokemon["Height"],pokemon["Weight"]])
    return tags

print(get_tags())

[['Parasect', 'sandybrown', 'pink', 'sienna', 'black', 'Bug', 1.0, 0.0, 0.0, 1.0, 29.5], ['Lumineon', 'darkslategray', 'paleturquoise', 'cadetblue', 'silver', 'Water', 4.0, 0.0, 0.0, 1.2, 24.0], ['Raikou', 'gray', 'gainsboro', 'gold', 'silver', 'Electric', 2.0, 1.0, 0.0, 1.9, 178.0], ['Comfey', 'silver', 'khaki', 'gray', 'indianred', 'Fairy', 7.0, 0.0, 0.0, 0.1, 0.3], ['Dedenne', 'burlywood', 'peru', 'darkslategray', 'black', 'Electric', 6.0, 0.0, 0.0, 0.2, 2.2], ['Pyroar', 'indianred', 'sandybrown', 'dimgray', 'silver', 'Fire', 6.0, 0.0, 0.0, 1.5, 81.5], ['Articuno', 'cornflowerblue', 'slategray', 'lavender', 'silver', 'Ice', 1.0, 1.0, 0.0, 1.7, 55.4], ['Magmortar', 'chocolate', 'sandybrown', 'darkolivegreen', 'silver', 'Fire', 4.0, 0.0, 0.0, 1.6, 68.0], ['Bulbasaur', 'powderblue', 'darkseagreen', 'cadetblue', 'dimgray', 'Grass', 1.0, 0.0, 0.0, 0.7, 6.9], ['Banette', 'darkgray', 'dimgray', 'darkslategray', 'darkgray', 'Ghost', 3.0, 0.0, 0.0, 1.1, 12.5], ['Staraptor', 'darkslategray', 